## Import the required packages to extract, clean and transform the data

In [82]:
# Import in-house modules from the 'utilities' package
from utilities import print_title, print_label
from utilities import save_data, load_data
from utilities import fetch_and_download_sp500_data, sp500_data_for_today
from utilities import split_dataset_by_date, clean_historical_data
from utilities import calculate_bollinger_bands, calculate_rsi, calculate_daily_volatility
from utilities import generate_trading_signals
from utilities import generate_directions

# Import datetime module
from datetime import datetime, timedelta

# Import libraries for data analysis and visualization
import numpy as np
import pandas as pd
import joblib
import pickle

## Fetch the SP500 data using a helper functions

In [46]:
historical_data = fetch_and_download_sp500_data(start_date="2007-01-01")
today_data      = sp500_data_for_today()

[*********************100%***********************]  501 of 501 completed


╔═══════════════════════════════════════════════════════════════╗
║            Download Report for S&P 500 `adj close`            ║
╠═══════════════════════════════════════════════════════════════╣
║ Total Requested Tickers:       |             501              ║
║ Total Downloaded Tickers:      |             501              ║
║ Requested Date Range:          |   2007-01-01 to 2024-10-30   ║
║ Downloaded Date Range:         |   2007-01-03 to 2024-10-29   ║
║                       S&P 500 Data Downloaded Successfully... ║
╚═══════════════════════════════════════════════════════════════╝
╔═══════════════════════════════════════════════════════════════╗
║      Download Report for S&P 500  `close` for 2024-10-29      ║
╠═══════════════════════════════════════════════════════════════╣
║ Total Requested Tickers:       |             501              ║
║ Requested Time:                |           15:59:00           ║
║ Downloaded Date:               |          2024-10-29          ║
║         

## Combine the data into a single dataset

In [47]:
df = pd.concat([historical_data, today_data], axis=0)

## Create DataFrame with features

In [48]:
upper_band,        lower_band         = calculate_bollinger_bands(df)
today_to_tomorrow, yesterday_to_today = generate_directions(df)

featured_df = pd.DataFrame({
  "Date":               np.repeat(df.index, len(df.columns)),
  "Ticker":             np.tile(df.columns, len(df)),
  "Adjusted Close":     df.values.flatten(),
  "Today to Tomorrow":  today_to_tomorrow.values.flatten(),
  "Yesterday to Today": yesterday_to_today.values.flatten(),
  "Next Day Close":     df.shift(-1).values.flatten(),
  "Previous Day Close": df.shift(1).values.flatten(),
  "Return":             df.pct_change().values.flatten(),
  "Volatility":         df.apply(calculate_daily_volatility).values.flatten(),
  "RSI":                df.apply(calculate_rsi).values.flatten(),
  "SMA_50":             df.rolling(window=50).mean().values.flatten(),
  "SMA_100":            df.rolling(window=100).mean().values.flatten(),
  "SMA_200":            df.rolling(window=200).mean().values.flatten(),
  "Upper Band":         upper_band.values.flatten(),
  "Lower Band":         lower_band.values.flatten(),
  "Support":            df.rolling(window=50).min().values.flatten(),
  "Resistance":         df.rolling(window=50).max().values.flatten(),
  "Action":             df.apply(generate_trading_signals).values.flatten()
})

# Replace inf values with NaN using an assignment
featured_df['Return'] = featured_df['Return'].replace([np.inf, -np.inf], np.nan)

# Convert to datetime (Without timezone)
featured_df['Date'] = pd.to_datetime(featured_df['Date'], utc=True).dt.date
featured_df['Date'] = pd.to_datetime(featured_df['Date'])

# Optimize the DataFrame by converting data types to more memory-efficient types
featured_df[['Ticker', 'Action']] = featured_df[['Ticker', 'Action']].astype('category')
float_cols = featured_df.select_dtypes(include=['float64']).columns
featured_df[float_cols] = featured_df[float_cols].astype('float32')

## Filter out backfilled data

In [49]:
# Filter rows from `2007-01-01` to `2007-12-31`
remove_dates_filter = (featured_df['Date'] >= "2007-01-01") & (featured_df['Date'] <= "2007-12-31")

# Remove the dates from the DataFrame
featured_df = featured_df.loc[~remove_dates_filter].copy()

## Split the data into training and testing data using a helper function
- We split the data on the previous date because the current date may not be available.

In [76]:
today      = datetime.now().date().strftime('%Y-%m-%d')
yesterday  = (datetime.now().date() - timedelta(days=1)).strftime('%Y-%m-%d')
split_date = today if featured_df["Date"].eq(today).any() else yesterday

historical_data, current_data = split_dataset_by_date(featured_df, split_date)

## Clean the data using a helper function

In [77]:
historical_data = clean_historical_data(historical_data)

## Prepare the prediction DataFrame
- Update the Date column to a datetime format
- Add Year, Month, Day, columns
- Set the index by the Date and Ticker columns

In [78]:
prediction_df = current_data.reset_index(drop=True)
prediction_df["Date"]  = pd.to_datetime(prediction_df["Date"])
prediction_df["Year"]  = prediction_df["Date"].dt.year
prediction_df["Month"] = prediction_df["Date"].dt.month
prediction_df["Day"]   = prediction_df["Date"].dt.day
prediction_df.set_index(["Date", "Ticker"], inplace=True)

## Drop low relevance columns determined by VIF and P-value analysis

In [79]:
select_target_to_drop   = ["Today to Tomorrow"]
select_features_to_drop = ["Action", "Next Day Close", "Previous Day Close", "Resistance", "Upper Band", "SMA_50", "SMA_200"]
log_select_to_drop      = ["Action", "Next Day Close", "Previous Day Close", "Resistance", "Upper Band", "SMA_50", "SMA_200",
                           "Next Day Close", "Adjusted Close", "Support", "Lower Band", "SMA_100"]

predict_data_clean     = prediction_df.drop(columns=[*select_features_to_drop, *select_target_to_drop])
log_predict_data_clean = prediction_df.drop(columns=[*log_select_to_drop, *select_target_to_drop])

## Import pre-trained scaling models and apply them to the data

In [95]:
X_scaler     = joblib.load("models/christian's_models/X_scaler.pkl")
X_scaler_log = joblib.load("models/christian's_models/X_scaler_log.pkl")
X_scaled     = X_scaler.transform(predict_data_clean)
X_scaled_log = X_scaler_log.transform(log_predict_data_clean)

## Import pre-trained prediction models

In [96]:
xbg_model           = pickle.load(open("models/christian's_models/clf_XGB_v2.pkl", "rb"))
random_forest_model = pickle.load(open("models/christian's_models/random_forest_classifier_v2.pkl", "rb"))
logistic_model      = pickle.load(open("models/christian's_models/logistic_regression_v1.pkl", "rb"))

## Predict the data using the pre-trained models

In [97]:
y_XGB = xbg_model.predict(X_scaled)
y_RFC = random_forest_model.predict(X_scaled)
y_LR  = logistic_model.predict(X_scaled_log)

# Change values where `0` is to `-1`
y_XGB[y_XGB == 0] = -1
y_RFC[y_RFC == 0] = -1
y_LR[y_LR == 0] = -1

y_XGB[:5], y_RFC[:5], y_LR[:5]

(array([1, 1, 1, 1, 1]),
 array([1, 1, 1, 1, 1], dtype=int32),
 array([1., 1., 1., 1., 1.]))

## Set up the output DataFrame

In [99]:
todays_data_predict = current_data[["Date", "Ticker", "Adjusted Close", "Today to Tomorrow"]]

# Change to integer
todays_data_predict["XGB Today to Tomorrow"] = y_XGB
todays_data_predict["RanFC Today to Tomorrow"] = y_RFC
todays_data_predict["Log_R Today to Tomorrow"] = y_LR.astype(int)

# Print the unique values of the predictions
print(todays_data_predict["XGB Today to Tomorrow"].unique())
print(todays_data_predict["RanFC Today to Tomorrow"].unique())
print(todays_data_predict["Log_R Today to Tomorrow"].unique())

todays_data_predict.head(50)

[ 1 -1]
[ 1 -1]
[ 1 -1]


/var/folders/v7/6x731rhn34s6b0csz38hy0rw0000gn/T/ipykernel_63859/1079433535.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  todays_data_predict["XGB Today to Tomorrow"] = y_XGB
/var/folders/v7/6x731rhn34s6b0csz38hy0rw0000gn/T/ipykernel_63859/1079433535.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  todays_data_predict["RanFC Today to Tomorrow"] = y_RFC
/var/folders/v7/6x731rhn34s6b0csz38hy0rw0000gn/T/ipykernel_63859/1079433535.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

,Date,Ticker,Adjusted Close,Today to Tomorrow,XGB Today to Tomorrow,RanFC Today to Tomorrow,Log_R Today to Tomorrow
0,2024-10-29,A,131.229996,1.0,1,1,1
1,2024-10-29,AAPL,233.669998,1.0,1,1,1
2,2024-10-29,ABBV,189.449997,1.0,1,1,1
3,2024-10-29,ABNB,137.779999,1.0,1,1,1
4,2024-10-29,ABT,113.400002,1.0,1,1,1
5,2024-10-29,ACGL,104.800003,1.0,1,1,1
6,2024-10-29,ACN,363.040009,1.0,1,1,1
7,2024-10-29,ADBE,485.390015,-1.0,1,1,1
8,2024-10-29,ADI,235.229996,1.0,1,1,1
9,2024-10-29,ADM,56.320000,1.0,1,1,1
